# RFE on full dataset

results are saved to the results_rfe_kappa and results_rfe_standard

In [4]:
import os
from copy import deepcopy
import warnings
import sys

# Scikit-learn and data handling
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from IPython.display import display, clear_output
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report, cohen_kappa_score, make_scorer
)
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# --- This script assumes your model files (moe_model.py, mop_model.py) are in the same directory

# --- Import your custom model files ---
from moe_model import MoE as MoE_raw, MLP as MoE_Expert
from mop_model import MoPModel as MoP_raw, MoPConfig

# --- Fix for MoE model to output logits ---
class MoE_Expert_Logits(MoE_Expert):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__(input_size, output_size, hidden_size)
        self.soft = nn.Identity()

# ===================================================================
# 1. SCikit-learn Wrapper Classes for PyTorch Models
# ===================================================================

class PyTorchWrapper(BaseEstimator, ClassifierMixin):
    """Base wrapper that creates a PyTorch model just-in-time for fitting."""
    def __init__(self, model_class, model_config, epochs=10, lr=0.01, batch_size=32):
        self.model_class = model_class
        self.model_config = model_config
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.model = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.classes_ = np.array([0, 1])

    def _create_model(self, n_features):
        config = deepcopy(self.model_config)
        if isinstance(config, MoPConfig):
            config.input_dim = n_features
            self.model = self.model_class(config).to(self.device)
        else:
            config['input_size'] = n_features
            model_instance = self.model_class(**config)
            model_instance.experts = nn.ModuleList([
                MoE_Expert_Logits(
                    input_size=n_features,
                    output_size=config['output_size'],
                    hidden_size=config['hidden_size']
                ) for _ in range(model_instance.num_experts)
            ])
            self.model = model_instance.to(self.device)

    def fit(self, X, y):
        if self.model is None or self._get_input_features() != X.shape[1]:
            self._create_model(X.shape[1])

        y_np = y.values if isinstance(y, (pd.Series, pd.DataFrame)) else y
        X_tensor = torch.FloatTensor(X).to(self.device)
        y_tensor = torch.LongTensor(y_np).to(self.device)
        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        self.model.train()
        for epoch in range(self.epochs):
            for X_batch, y_batch in loader:
                optimizer.zero_grad()
                y_pred = self._forward(X_batch)
                loss = criterion(y_pred, y_batch)
                aux_loss = self._get_aux_loss(X_batch)
                if aux_loss is not None:
                    loss += aux_loss
                loss.backward()
                optimizer.step()

        if hasattr(self.model, 'input_layer') and hasattr(self.model.input_layer, 'weight'):
            weights = self.model.input_layer.weight.data.cpu().numpy()
            self.feature_importances_ = np.abs(weights).sum(axis=0)
        elif hasattr(self.model, 'w_gate'):
            weights = self.model.w_gate.data.cpu().numpy()
            self.feature_importances_ = np.abs(weights).sum(axis=1)
        else:
            try:
                weights = self.model.experts[0].fc1.weight.data.cpu().numpy()
                self.feature_importances_ = np.abs(weights).sum(axis=0)
            except:
                 raise AttributeError("Wrapped model needs a way to determine feature importance.")
        return self

    def predict(self, X):
        if self.model is None: self._create_model(X.shape[1])
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(self.device)
            y_pred = self._forward(X_tensor)
            _, predicted = torch.max(y_pred.data, 1)
        return predicted.cpu().numpy()

    def predict_proba(self, X):
        if self.model is None: self._create_model(X.shape[1])
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(self.device)
            y_pred = self._forward(X_tensor)
            probas = nn.functional.softmax(y_pred, dim=1)
        return probas.cpu().numpy()

    def _get_input_features(self):
        if self.model is None: return -1
        if hasattr(self.model, 'input_layer'): return self.model.input_layer.in_features
        if hasattr(self.model, 'w_gate'): return self.model.w_gate.shape[0]
        return -1

    def _forward(self, X_batch): raise NotImplementedError
    def _get_aux_loss(self, X_batch): return None

class MoEWrapper(PyTorchWrapper):
    def _forward(self, X_batch):
        y_pred, _ = self.model(X_batch)
        return y_pred
    def _get_aux_loss(self, X_batch):
        _, aux_loss = self.model(X_batch)
        return aux_loss

class MoPWrapper(PyTorchWrapper):
    def _forward(self, X_batch):
        y_pred, _, _ = self.model(X_batch.unsqueeze(1))
        return y_pred.squeeze(1)

# ===================================================================
# 2. RFE Evaluation Framework
# ===================================================================

def evaluate_model_scoring(X_train, y_train, X_val, y_val, X_test, y_test, feature_names, name, base_est, scoring, cv):
    est_copy = deepcopy(base_est)
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('fs', RFECV(estimator=est_copy, step=1, cv=cv, scoring=scoring, n_jobs=-1, min_features_to_select=5)),
        ('clf', deepcopy(base_est))
    ])
    pipe.fit(X_train, y_train)
    support = pipe.named_steps['fs'].support_
    selected = np.array(feature_names)[support].tolist()
    
    def compute_metrics(X, y, dataset):
        y_pred = pipe.predict(X)
        y_score = pipe.predict_proba(X)[:, 1]
        auc = roc_auc_score(y, y_score)
        rep = classification_report(y, y_pred, output_dict=True, zero_division=0).get('1', {})
        return {
            'Dataset': dataset, 'Scoring': scoring if isinstance(scoring, str) else 'cohen_kappa', 'Model': name,
            'Accuracy': accuracy_score(y, y_pred), 'ROC-AUC': auc,
            'Precision': rep.get('precision'), 'Recall': rep.get('recall'),
            'F1 Score': rep.get('f1-score'), 'Cohen Kappa': cohen_kappa_score(y, y_pred),
            'n_features': len(selected), 'features': selected
        }
    return (compute_metrics(X_train, y_train, 'Train'),
            compute_metrics(X_val, y_val, 'Validation'),
            compute_metrics(X_test, y_test, 'Test'))

def run_rfe_experiment(X_train, y_train, X_val, y_val, X_test, y_test,
                       feature_names, estimators, cv, scorings, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    scoring_items = scorings.items() if isinstance(scorings, dict) else [(s, s) for s in scorings]
    
    tasks = [
        (name, model, scoring_obj)
        for scoring_name, scoring_obj in scoring_items
        for name, model in estimators.items()
    ]
    
    all_results = []

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i, (name, model, scoring_obj) in enumerate(tasks):
            scoring_name = scoring_obj if isinstance(scoring_obj, str) else 'cohen_kappa'
            print(f"✅ Progress: Starting job {i + 1} of {len(tasks)} for run '{output_dir}'")
            print(f"➡️  Now running RFE for model '{name}' with scoring '{scoring_name}'. This may take several minutes...")
            
            result = evaluate_model_scoring(
                X_train, y_train, X_val, y_val, X_test, y_test,
                feature_names, name, model, scoring_obj, cv
            )
            all_results.append(result)
            print(f"🏁 Finished job {i + 1} of {len(tasks)}.")

    # --- All jobs are complete, now process and print the final tables ---
    train_records, val_records, test_records = zip(*all_results)
    train_df = pd.DataFrame(list(train_records))
    val_df = pd.DataFrame(list(val_records))
    test_df = pd.DataFrame(list(test_records))
    
    # --- Save the complete results to CSV files ---
    train_df.to_csv(os.path.join(output_dir, 'train_results.csv'), index=False)
    val_df.to_csv(os.path.join(output_dir, 'validation_results.csv'), index=False)
    test_df.to_csv(os.path.join(output_dir, 'test_results.csv'), index=False)
    print(f"\n💾 Complete results saved to '{output_dir}' directory.")
    
    # --- Display final tables ---
    clear_output(wait=True)
    print(f"✅ All jobs complete for run '{output_dir}'")
    
    print("\n✅ Final Training Set Performance")
    display(train_df.style.set_caption(f"Training Set Performance ({output_dir})"))
    
    print("\n✅ Final Validation Set Performance")
    display(val_df.style.set_caption(f"Validation Set Performance ({output_dir})"))
    
    print("\n✅ Final Test Set Performance")
    display(test_df.style.set_caption(f"Test Set Performance ({output_dir})"))
    
    return train_df, val_df, test_df

# ===================================================================
# Main Execution Block
# ===================================================================

if __name__ == '__main__':
    try:
        print("🔹 Loading and preparing data...")
        df = pd.read_csv('uk_biobank_dataset.csv', low_memory=False)
        X, y = df.drop(columns=['Dementia Status']), df['Dementia Status']
        feature_names = X.columns.tolist()
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
        X_train, y_train = X_train.values, y_train.values
        X_val, y_val = X_val.values, y_val.values
        X_test, y_test = X_test.values, y_test.values
        print(f"Data loaded successfully. Train shape: {X_train.shape}")
        n_features = X_train.shape[1]
    except FileNotFoundError:
        print("\n⚠️ ERROR: Data file not found. Please update the path.")
        exit()

    mop_config = MoPConfig(input_dim=n_features, output_dim=2, intermediate_dim=32, layers=["0,8,16"])
    moe_config = {'input_size': n_features, 'output_size': 2, 'num_experts': 4, 'hidden_size': 16, 'k': 2}
    estimators = {
        'MoP': MoPWrapper(MoP_raw, mop_config, epochs=5), # Reduced epochs for faster RFE
        'MoE': MoEWrapper(MoE_raw, moe_config, epochs=5)
    }
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=42)

    # --- Run 1: Standard Scoring Metrics ---
    run_rfe_experiment(
        X_train, y_train, X_val, y_val, X_test, y_test,
        feature_names, estimators, cv,
        scorings=['accuracy', 'f1', 'precision', 'roc_auc'], # All metrics included
        output_dir='results_rfe_standard'
    )
    
    # --- Run 2: Cohen's Kappa Scoring ---
    cohen_kappa_scorer = make_scorer(cohen_kappa_score)
    run_rfe_experiment(
        X_train, y_train, X_val, y_val, X_test, y_test,
        feature_names, estimators, cv,
        scorings={'cohen_kappa': cohen_kappa_scorer},
        output_dir='results_rfe_kappa'
    )


✅ All jobs complete for run 'results_rfe_kappa'

✅ Final Training Set Performance


,Dataset,Scoring,Model,Accuracy,ROC-AUC,Precision,Recall,F1 Score,Cohen Kappa,n_features,features
0,Train,cohen_kappa,MoP,0.774865,0.867348,0.753482,0.824695,0.787482,0.549171,46,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Townsend deprivation index at recruitment', 'Number in household | Instance 0', 'Has_Heart_attack', 'Has_Angina', 'Has_Stroke', 'Has_High_blood_pressure', 'Has_Any_Vascular_Heart_Problem', 'Sleep duration | Instance 0', 'Number of days/week of vigorous physical activity 10+ minutes | Instance 0', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'Cholesterol | Instance 0', 'Triglycerides | Instance 0', 'C-reactive protein | Instance 0', 'Glucose | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Creatinine | Instance 0', 'Pretest_RSA_ln_ms2_Inst0', 'Activity_RSA_ln_ms2_Inst0', 'Sex_Female', 'Sex_Male', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Current', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Current', 'Alcohol drinker status | Instance 0_Previous', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Never', 'Alcohol intake frequency. | Instance 0_Once or twice a week', 'Alcohol intake frequency. | Instance 0_One to three times a month', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Bipolar I Disorder', 'Bipolar and major depression status | Instance 0_No Bipolar or Depression', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Worrier / anxious feelings | Instance 0_Do not know', 'Worrier / anxious feelings | Instance 0_Prefer not to answer', 'Worrier / anxious feelings | Instance 0_Yes', 'Hand grip strength | Instance 0', 'Gate_speed_slow']"
1,Train,cohen_kappa,MoE,0.815729,0.900422,0.781377,0.882622,0.828919,0.630848,25,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Has_Heart_attack', 'Has_Stroke', 'Has_High_blood_pressure', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'C-reactive protein | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Average_HR_bpm_Inst0', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Probable recurrent major depression (moderate) | Instance 0_Yes', 'Worrier / anxious feelings | Instance 0_No', 'Gate_speed_slow']"



✅ Final Validation Set Performance


,Dataset,Scoring,Model,Accuracy,ROC-AUC,Precision,Recall,F1 Score,Cohen Kappa,n_features,features
0,Validation,cohen_kappa,MoP,0.730216,0.789020,0.720000,0.765957,0.742268,0.459817,46,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Townsend deprivation index at recruitment', 'Number in household | Instance 0', 'Has_Heart_attack', 'Has_Angina', 'Has_Stroke', 'Has_High_blood_pressure', 'Has_Any_Vascular_Heart_Problem', 'Sleep duration | Instance 0', 'Number of days/week of vigorous physical activity 10+ minutes | Instance 0', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'Cholesterol | Instance 0', 'Triglycerides | Instance 0', 'C-reactive protein | Instance 0', 'Glucose | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Creatinine | Instance 0', 'Pretest_RSA_ln_ms2_Inst0', 'Activity_RSA_ln_ms2_Inst0', 'Sex_Female', 'Sex_Male', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Current', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Current', 'Alcohol drinker status | Instance 0_Previous', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Never', 'Alcohol intake frequency. | Instance 0_Once or twice a week', 'Alcohol intake frequency. | Instance 0_One to three times a month', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Bipolar I Disorder', 'Bipolar and major depression status | Instance 0_No Bipolar or Depression', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Worrier / anxious feelings | Instance 0_Do not know', 'Worrier / anxious feelings | Instance 0_Prefer not to answer', 'Worrier / anxious feelings | Instance 0_Yes', 'Hand grip strength | Instance 0', 'Gate_speed_slow']"
1,Validation,cohen_kappa,MoE,0.694245,0.768339,0.675000,0.765957,0.717608,0.387157,25,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Has_Heart_attack', 'Has_Stroke', 'Has_High_blood_pressure', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'C-reactive protein | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Average_HR_bpm_Inst0', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Probable recurrent major depression (moderate) | Instance 0_Yes', 'Worrier / anxious feelings | Instance 0_No', 'Gate_speed_slow']"



✅ Final Test Set Performance


,Dataset,Scoring,Model,Accuracy,ROC-AUC,Precision,Recall,F1 Score,Cohen Kappa,n_features,features
0,Test,cohen_kappa,MoP,0.741007,0.826579,0.729730,0.771429,0.750000,0.481773,46,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Townsend deprivation index at recruitment', 'Number in household | Instance 0', 'Has_Heart_attack', 'Has_Angina', 'Has_Stroke', 'Has_High_blood_pressure', 'Has_Any_Vascular_Heart_Problem', 'Sleep duration | Instance 0', 'Number of days/week of vigorous physical activity 10+ minutes | Instance 0', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'Cholesterol | Instance 0', 'Triglycerides | Instance 0', 'C-reactive protein | Instance 0', 'Glucose | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Creatinine | Instance 0', 'Pretest_RSA_ln_ms2_Inst0', 'Activity_RSA_ln_ms2_Inst0', 'Sex_Female', 'Sex_Male', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Current', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Current', 'Alcohol drinker status | Instance 0_Previous', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Never', 'Alcohol intake frequency. | Instance 0_Once or twice a week', 'Alcohol intake frequency. | Instance 0_One to three times a month', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Bipolar I Disorder', 'Bipolar and major depression status | Instance 0_No Bipolar or Depression', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Worrier / anxious feelings | Instance 0_Do not know', 'Worrier / anxious feelings | Instance 0_Prefer not to answer', 'Worrier / anxious feelings | Instance 0_Yes', 'Hand grip strength | Instance 0', 'Gate_speed_slow']"
1,Test,cohen_kappa,MoE,0.737410,0.795600,0.727891,0.764286,0.745645,0.474603,25,"['Age at recruitment', 'Year of birth', ""Standard PRS for alzheimer's disease (AD)"", 'Has_Heart_attack', 'Has_Stroke', 'Has_High_blood_pressure', 'Mean time to correctly identify matches | Instance 0', 'Fluid intelligence score | Instance 0', 'C-reactive protein | Instance 0', 'Glycated haemoglobin (HbA1c) | Instance 0', 'Average_HR_bpm_Inst0', 'Diabetes_Status_Do not know', 'Diabetes_Status_No', 'Diabetes_Status_Yes', 'Smoking status | Instance 0_Never', 'Smoking status | Instance 0_Previous', 'Alcohol drinker status | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Daily or almost daily', 'Alcohol intake frequency. | Instance 0_Prefer not to answer', 'Alcohol intake frequency. | Instance 0_Special occasions only', 'Alcohol intake frequency. | Instance 0_Three or four times a week', 'Bipolar and major depression status | Instance 0_Single Probable major depression episode', 'Probable recurrent major depression (moderate) | Instance 0_Yes', 'Worrier / anxious feelings | Instance 0_No', 'Gate_speed_slow']"
